In [2]:
import pandas as pd
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
import os
import pandas as pd
from google.colab import drive



# Path to the zip file
zip_file_path = '/content/drive/MyDrive/image classification/places365_contents.zip'

# Directory to extract the contents
extracted_dir = 'data_contents/'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Create a list to store dataframe rows
data = []

# Traverse through the extracted directory to create the dataframe
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            label_name = os.path.basename(root)
            data.append({'path': file_path, 'name': file, 'label': label_name})

# Create a DataFrame
df = pd.DataFrame(data)

# Adjust path format for 'path name' column
df['path name'] = 'data_contents/' + df['label'] + '/' + df['name']

# Rearrange columns
df = df[['path name', 'name', 'label']]

# Print the DataFrame
print(df)

                                    path name          name        label
0      data_contents/art_gallery/00000275.jpg  00000275.jpg  art_gallery
1      data_contents/art_gallery/00004531.jpg  00004531.jpg  art_gallery
2      data_contents/art_gallery/00000609.jpg  00000609.jpg  art_gallery
3      data_contents/art_gallery/00003223.jpg  00003223.jpg  art_gallery
4      data_contents/art_gallery/00001162.jpg  00001162.jpg  art_gallery
...                                       ...           ...          ...
62515   data_contents/bank_vault/00000842.jpg  00000842.jpg   bank_vault
62516   data_contents/bank_vault/00002841.jpg  00002841.jpg   bank_vault
62517   data_contents/bank_vault/00002416.jpg  00002416.jpg   bank_vault
62518   data_contents/bank_vault/00001035.jpg  00001035.jpg   bank_vault
62519   data_contents/bank_vault/00002327.jpg  00002327.jpg   bank_vault

[62520 rows x 3 columns]


In [5]:
df['label'].value_counts()

art_gallery             5000
hospital_room           5000
childs_room             5000
airplane_cabin          5000
bakery-shop             5000
dining_hall             5000
amusement_arcade        5000
swimming_pool-indoor    5000
garage-indoor           5000
ballroom                5000
locker_room             5000
sauna                   4020
bank_vault              3500
Name: label, dtype: int64

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Flatten, Dense
from tensorflow.keras.models import Sequential

def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(64, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(128, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(128, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(256, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(256, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (112, 112, 3)  # Change this according to your data
num_classes = 13  # Change this to the correct number of classes

# Create the model
model = create_model(input_shape, num_classes)

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 56, 56, 64)        256       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 56, 56, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 28, 28, 64)        0

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'df'
# Split the DataFrame into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Data augmentation: random rotations
    width_shift_range=0.2,  # Data augmentation: random width shifts
    height_shift_range=0.2,  # Data augmentation: random height shifts
    horizontal_flip=True,  # Data augmentation: horizontal flips
)
batch_size = 32
# Load and preprocess training data from DataFrame
train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='path name',  # Column containing image paths
    y_col='label',  # Column containing labels
    target_size=(112, 112),  # Resize images to match model input size
    batch_size=batch_size,
    class_mode='categorical',  # Categorical labels for multi-class classification
)

# Load and preprocess validation data from DataFrame
validation_generator = datagen.flow_from_dataframe(
    val_df,
    x_col='path name',
    y_col='label',
    target_size=(112, 112),
    batch_size=batch_size,
    class_mode='categorical',
)


label_map = (train_generator.class_indices)
print(label_map)

Found 50016 validated image filenames belonging to 13 classes.
Found 12504 validated image filenames belonging to 13 classes.
{'airplane_cabin': 0, 'amusement_arcade': 1, 'art_gallery': 2, 'bakery-shop': 3, 'ballroom': 4, 'bank_vault': 5, 'childs_room': 6, 'dining_hall': 7, 'garage-indoor': 8, 'hospital_room': 9, 'locker_room': 10, 'sauna': 11, 'swimming_pool-indoor': 12}


In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/environment recognition/model_100_epochs.h5')

In [ ]:
from keras.callbacks import ModelCheckpoint
import os

epochs = 100
save_interval = 5  # Save after every epoch
model_save_dir = "/content/drive/MyDrive/environment recognition"  # Directory to save checkpoints

# Create the directory if it doesn't exist
os.makedirs(model_save_dir, exist_ok=True)

# Define a custom callback class that inherits from ModelCheckpoint
class CustomModelCheckpoint(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        if epoch >= 30:  # Start saving from epoch 30
            super().on_epoch_end(epoch + 30, logs)
#/content/drive/MyDrive/environment recognition/model_30_epochs.h5
checkpoint = CustomModelCheckpoint(
    filepath=os.path.join(model_save_dir, "model_epoch{epoch+30:02d}.h5"),  # Save format
    save_best_only=False,
    save_weights_only=False,
    period=save_interval
)

# Train the model with the callback
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint]
)


Epoch 1/100
1563/1563 [==============================] - 343s 211ms/step - loss: 0.3178 - accuracy: 0.9029 - val_loss: 0.3011 - val_accuracy: 0.9054
Epoch 2/100
1563/1563 [==============================] - 331s 212ms/step - loss: 0.2915 - accuracy: 0.9080 - val_loss: 0.3866 - val_accuracy: 0.8743
Epoch 3/100
1563/1563 [==============================] - 320s 205ms/step - loss: 0.2800 - accuracy: 0.9093 - val_loss: 0.2811 - val_accuracy: 0.9124
Epoch 4/100
1563/1563 [==============================] - 321s 206ms/step - loss: 0.2739 - accuracy: 0.9118 - val_loss: 0.3383 - val_accuracy: 0.8928
Epoch 5/100
1563/1563 [==============================] - 292s 187ms/step - loss: 0.2686 - accuracy: 0.9123 - val_loss: 0.3718 - val_accuracy: 0.8837
Epoch 6/100
1563/1563 [==============================] - 321s 205ms/step - loss: 0.2665 - accuracy: 0.9136 - val_loss: 0.3908 - val_accuracy: 0.8758
Epoch 7/100
1563/1563 [==============================] - 291s 186ms/step - loss: 0.2571 - accuracy: 0.9146

KeyboardInterrupt: ignored

In [ ]:
# Train the model
epochs = 10  # Adjust as needed
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
1563/1563 [==============================] - 209s 134ms/step - loss: 0.2128 - accuracy: 0.9275 - val_loss: 0.7799 - val_accuracy: 0.8011
Epoch 2/10
1563/1563 [==============================] - 211s 135ms/step - loss: 0.2110 - accuracy: 0.9273 - val_loss: 0.6955 - val_accuracy: 0.8171
Epoch 3/10
1563/1563 [==============================] - 210s 135ms/step - loss: 0.2115 - accuracy: 0.9277 - val_loss: 0.8335 - val_accuracy: 0.7897
Epoch 4/10
1563/1563 [==============================] - 210s 134ms/step - loss: 0.2126 - accuracy: 0.9268 - val_loss: 0.7351 - val_accuracy: 0.8079
Epoch 5/10
1563/1563 [==============================] - 207s 133ms/step - loss: 0.2156 - accuracy: 0.9262 - val_loss: 0.7012 - val_accuracy: 0.8189
Epoch 6/10
1563/1563 [==============================] - 207s 132ms/step - loss: 0.2031 - accuracy: 0.9309 - val_loss: 0.7376 - val_accuracy: 0.8075
Epoch 7/10
1563/1563 [==============================] - 208s 133ms/step - loss: 0.2097 - accuracy: 0.9274 - val_

In [ ]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

# Save the model
model.save('/content/drive/MyDrive/environment recognition/model_200_epochs.h5')  # Adjust the filename as needed

In [ ]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

# Save the model
model.save('/content/drive/MyDrive/environment recognition/model_200_epochs.h5')  # Adjust the filename as needed

391/391 [==============================] - 44s 112ms/step - loss: 0.0371 - accuracy: 0.9876
Validation Loss: 0.03714356943964958
Validation Accuracy: 0.9876039624214172
